In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model

In [34]:
heroinUsers = pd.read_excel('./tables/drugOffenses/heroinUsers.xlsx', 
                            usecols=[i for i in range(15)], nrows=30) # for a clean dataframe
heroinSuppliers = pd.read_excel('./tables/drugOffenses/heroinSuppliers.xlsx', 
                            usecols=[i for i in range(15)], nrows=30)

In [35]:
target_country = 'Austria' # target country for model

In [36]:
df = heroinSuppliers[heroinSuppliers['Country'] == target_country].T[1:]
df.columns = ["Suppliers " + target_country]

for country in list(heroinSuppliers['Country']):
    if country != target_country:
        joiner = heroinSuppliers[heroinSuppliers['Country'] == country].T[1:]
        joiner.columns = ["Suppliers " + country]
        df = df.join(joiner)

In [37]:
# users in target country
joiner = heroinUsers[heroinUsers['Country'] == target_country].T[1:]
joiner.columns = ["Users " + target_country]
df = df.join(joiner)

In [32]:
# i dont like this, doesnt work because it gives the mean value a lot on nan
for country in df.columns:
    if df[country].isnull().sum() == len(df[country]): # checks if all of its values are nan
        df = df.drop(country, axis = 1) # if all are NAN drop the column
    else:
        df[country] = df[country].fillna(df[country].mean()) # and if all are not NAN it fills it with the mean value
                                                # to at least have something

In [60]:
df_withoutUsers = df.loc[:, df.columns != 'Users Austria'].fillna(0)
df['Years'] = [i for i in range(2004, 2018)][::-1]
df["Users Austria"] = df["Users Austria"].fillna(0)

In [54]:
reg = linear_model.LinearRegression()
reg.fit(df_withoutUsers, df["Users Austria"])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [61]:
df

,Suppliers Austria,Suppliers Belgium,Suppliers Bulgaria,Suppliers Croatia,Suppliers Cyprus,Suppliers Czechia,Suppliers Denmark,Suppliers Estonia,Suppliers Finland,Suppliers France,...,Suppliers Portugal,Suppliers Romania,Suppliers Slovakia,Suppliers Slovenia,Suppliers Spain,Suppliers Sweden,Suppliers Turkey,Suppliers United Kingdom,Users Austria,Years
2017,801,692,NaN,168,NaN,46,NaN,NaN,NaN,NaN,...,492,NaN,13,NaN,1246,NaN,3701,NaN,1838.0,2017
2016,617,815,NaN,98,NaN,NaN,NaN,NaN,NaN,NaN,...,774,NaN,NaN,80,1477,NaN,2182,5560,1535.0,2016
2015,560,846,NaN,144,2,38,NaN,NaN,NaN,NaN,...,763,NaN,16,80,1438,NaN,3423,4708,1018.0,2015
2014,467,963,106,41,0,29,NaN,NaN,NaN,NaN,...,690,NaN,22,55,1602,NaN,1740,4962,1134.0,2014
2013,293,918,274,63,2,54,NaN,NaN,NaN,NaN,...,153,NaN,32,105,1371,NaN,1293,5027,975.0,2013
2012,343,1087,192,103,3,59,NaN,NaN,NaN,NaN,...,214,NaN,29,142,1408,NaN,1099,5069,1110.0,2012
2011,561,1181,242,216,5,50,NaN,NaN,NaN,NaN,...,248,NaN,55,218,1030,NaN,839,5780,1875.0,2011
2010,667,1282,293,177,10,78,NaN,NaN,NaN,NaN,...,303,NaN,40,279,1518,NaN,NaN,6770,2887.0,2010
2009,601,1013,321,232,1,103,NaN,NaN,NaN,1333,...,336,NaN,32,255,1356,NaN,NaN,6356,2453.0,2009
2008,593,1229,337,1350,NaN,124,NaN,NaN,NaN,NaN,...,255,NaN,49,272,1558,NaN,NaN,7417,2153.0,2008


## simple regression on Austria based on years and suppliers in Austria

In [74]:
target_country = 'Austria'
df_simple = heroinSuppliers[heroinSuppliers['Country'] == target_country].T[1:]
df_simple.columns = ["Suppliers " + target_country]

joiner = heroinUsers[heroinUsers['Country'] == target_country].T[1:]
joiner.columns = ["Users " + target_country]
df_simple = df_simple.join(joiner)
df_simple = df_simple.drop(2004)
df_simple['Years'] = [i for i in range(2005, 2018)][::-1]

In [76]:
reg_simple = linear_model.LinearRegression()
reg_simple.fit(df_simple[['Years','Suppliers Austria']], df_simple["Users Austria"])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [92]:
suppliers = df_simple['Suppliers Austria'].mean()
for year in range(2018, 2025):
    users = reg_simple.predict([[year, suppliers]])[0]
    print("In the year " + str(year) + " the model has predicted " + 
          str(users) + " users in Austria on " + str(suppliers) + " suppliers")
print("====================")
suppliers = 700
for year in range(2018, 2025):
    users = reg_simple.predict([[year, suppliers]])[0]
    print("In the year " + str(year) + " the model has predicted " + 
          str(users) + " users in Austria on " + str(suppliers) + " suppliers")

In the year 2018 the model has predicted 1020.3006127455155 users in Austria on 586.5384615384615 suppliers
In the year 2019 the model has predicted 880.9919090718031 users in Austria on 586.5384615384615 suppliers
In the year 2020 the model has predicted 741.6832053980906 users in Austria on 586.5384615384615 suppliers
In the year 2021 the model has predicted 602.3745017243782 users in Austria on 586.5384615384615 suppliers
In the year 2022 the model has predicted 463.06579805066576 users in Austria on 586.5384615384615 suppliers
In the year 2023 the model has predicted 323.7570943769533 users in Austria on 586.5384615384615 suppliers
In the year 2024 the model has predicted 184.44839070324088 users in Austria on 586.5384615384615 suppliers
In the year 2018 the model has predicted 1379.8000689211185 users in Austria on 700 suppliers
In the year 2019 the model has predicted 1240.491365247406 users in Austria on 700 suppliers
In the year 2020 the model has predicted 1101.1826615736936 u

In [75]:
df_simple

,Suppliers Austria,Users Austria,Years
2017,801,1838,2017
2016,617,1535,2016
2015,560,1018,2015
2014,467,1134,2014
2013,293,975,2013
2012,343,1110,2012
2011,561,1875,2011
2010,667,2887,2010
2009,601,2453,2009
2008,593,2153,2008
